<a href="https://colab.research.google.com/github/gokulbot/StartingWithPython-ai1-fall2019/blob/master/mnist_keras_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing

In [5]:
!pip install wandb

     |████████████████████████████████| 184kB 40.1MB/s 
  Created wheel for watchdog: filename=watchdog-0.9.0-cp36-none-any.whl size=73652 sha256=1dc051a739ad5cb50352e279737cde1634e690d547076c2c09858608c837ff98
  Stored in directory: /root/.cache/pip/wheels/61/1d/d0/04cfe495619be2095eb8d89a31c42adb4e42b76495bc8f784c
  Created wheel for shortuuid: filename=shortuuid-0.5.0-cp36-none-any.whl size=5499 sha256=14f298d2bcf6bfeeef0c846d86874a9b10c68f8105a440a3f79513e2da4133f5
  Stored in directory: /root/.cache/pip/wheels/3f/eb/fd/69e5177f67b505e44acbd1aedfbe44b91768ee0c4cd5636576
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=208ddab4fd95cd3fb07ef09f16c1dbd9f350e76ae1f5688575b88ec046dce333
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for gql: filename=gql-0.1.0-cp36-none-any.whl size=5541 sha256=1e1183de26f633d672b3a814f913c2c07923071cb7ebba73f8744a2d8ab1ce9a
  Stor

In [0]:
# from lukas/mlclass
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.utils import np_utils
from wandb.keras import WandbCallback
import wandb
import os



In [7]:
run = wandb.init()

<IPython.core.display.Javascript object>

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Config

In [0]:
# 
config = run.config
# class Config:
#   pass
# config = Config()
config.first_layer_convs = 32
config.first_layer_conv_width = 3
config.first_layer_conv_height = 3
config.dropout = 0.2
config.dense_layer_size = 100
config.img_width = 28
config.img_height = 28
config.epochs = 10

## Setup

In [0]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.astype('float32')
X_train /= 255.
X_test = X_test.astype('float32')
X_test /= 255.



In [10]:
X_train.shape, y_test.shape

((60000, 28, 28), (10000,))

In [0]:
# reshape input data
X_train = X_train.reshape(
    X_train.shape[0], config.img_width, config.img_height, 1)
X_test = X_test.reshape(
    X_test.shape[0], config.img_width, config.img_height, 1)

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
print(y_train.shape)
num_classes = y_test.shape[1]
labels = range(10)

## Model Building

In [0]:
# build model
model = Sequential()
model.add(Conv2D(config.first_layer_convs,
                 (config.first_layer_conv_width, config.first_layer_conv_height),
                 input_shape=(28, 28, 1),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.4))
model.add(Conv2D(64,
                 (3, 3),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(config.dense_layer_size, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])
model.summary()


In [0]:
%%wandb
## Model Running
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
          epochs=config.epochs,
          callbacks=[WandbCallback(data_type="image", save_model=False)])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 24s 400us/step - loss: 0.1787 - acc: 0.9447 - val_loss: 0.0500 - val_acc: 0.9844
Epoch 2/10
60000/60000 [==============================] - 17s 285us/step - loss: 0.0712 - acc: 0.9776 - val_loss: 0.0335 - val_acc: 0.9882
Epoch 3/10
60000/60000 [==============================] - 17s 279us/step - loss: 0.0526 - acc: 0.9834 - val_loss: 0.0266 - val_acc: 0.9903
Epoch 4/10
60000/60000 [==============================] - 16s 273us/step - loss: 0.0443 - acc: 0.9863 - val_loss: 0.0259 - val_acc: 0.9905
Epoch 5/10
60000/60000 [==============================] - 17s 277us/step - loss: 0.0390 - acc: 0.9876 - val_loss: 0.0322 - val_acc: 0.9888
Epoch 6/10
60000/60000 [==============================] - 16s 273us/step - loss: 0.0343 - acc: 0.9889 - val_loss: 0.0278 - val_acc: 0.9916
Epoch 7/10
60000/60000

In [0]:
import matplotlib.pyplot as plt
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()